In [1]:
from metric.compute_metric import compute_clear
from metric.read_file import read_file
from utils import make_pred, save_data, iter_frame, body_barycenter, head_barycenter
from utils import Point
from utils import Corp
import numpy as np
PATH = "data/Scenario04-01/"

In [2]:
# data_ref = read_file(PATH+"groundTruth.txt")[630:680]
# row_data = np.loadtxt(PATH+"detections.txt")[630:680]
data_ref = read_file(PATH+"groundTruth.txt")
row_data = np.loadtxt(PATH+"detections.txt")

print(data_ref.shape)
print(row_data.shape)

(13273, 39)
(13255, 37)


On essaie une methode ou l'on attribue les identité en fonction de la proximitée des tetes a la frame précedent.

A amméliorer (methode hongroise, inertie ... ) 

In [3]:
from random import randint, random

def make_alea(proba_trans):
    def alea_par_func(data):
        save_pos = dict()
        parle = 0
        for frame in iter_frame(data):
            nb_personne = frame.shape[0]

            if random() < proba_trans:
                parle = randint(0, nb_personne)

            save_pos = head_assignation(frame, save_pos)
            for i, _ in enumerate(frame):
                cp = Corp(_)
                yield (save_pos[cp], save_pos[cp] == parle)
    return alea_par_func


def head_assignation(frame, oldPos):
    to_assin = []
    new_pos = []
    rep = dict()
    for line in frame:
        to_assin.append(Corp(line))
        new_pos.append(0)

    comb = []
    for i in oldPos.keys():
        for j in to_assin:
            comb.append((i, j))

    comb.sort(key=dist)
    old_pos_traite = []
    new_pos_traite = []
    
    
    for (pt_old, pt_new) in comb:
        if (pt_old not in old_pos_traite) and (pt_new not in new_pos_traite):
            old_pos_traite.append(pt_old)
            new_pos_traite.append(pt_new)
            rep[pt_new] = oldPos[pt_old]
            pt_new.nb_of_use = pt_old.nb_of_use + 1

    for elm in to_assin:
        if elm not in new_pos_traite:
            if len(rep) == 0:
                rep[elm] = 1
            else: 
                rep[elm] = max(rep.values()) + 1
                # print ("\n")
                # print("New element ! ! ! " + str(rep))
                # print("oldpos : " + str(oldPos))
                # print(to_assin)
                # print("Frame : " + str(frame))
            new_pos_traite.append(elm)
            
    for elm in oldPos.keys():
        if elm not in old_pos_traite and elm.nb_of_use > 5:
            rep[elm] = oldPos[elm]
            old_pos_traite.append(elm)       

    return rep


def dist((A, B)):
    return A.dist(B)

res = make_pred(row_data, make_alea(0.001))
print(res.shape)
print(compute_clear(data_ref, res, 1))


(13231, 39)


(0.9787506593323789, 0.0, 0.6461211477151966, 0.6274583678697913)


In [4]:
from munkres import Munkres, print_matrix

def make_alea(proba_trans):
    def alea_par_func(data):
        save_pos = dict()
        parle = 0
        for frame in iter_frame(data):
            nb_personne = frame.shape[0]

            if random() < proba_trans:
                parle = randint(0, nb_personne)

            save_pos = head_assignation(frame, save_pos)

            for i, _ in enumerate(frame):
                cp = Corp(_)
                # if save_pos[cp] == 5:
                    # print(save_pos)
                yield (save_pos[cp], save_pos[cp] == parle)
    return alea_par_func


def head_assignation(frame, oldPos):
    to_assin = []
    new_pos = []
    rep = dict()
    for line in frame:
        to_assin.append(Corp(line))
        new_pos.append(0)
        
    new_pos_traite = []
    old_pos_traite = []
    
    m = Munkres()
    matrice = []
    old_pos_order = oldPos.keys()
    for old in oldPos:
        l =[]
        for new in to_assin:
            l.append(dist((old, new)))
        matrice.append(l)
    # print("old pos oder" + str(old_pos_order))
    # print("to_assin" + str(to_assin))
    # print(matrice)
    # 
    if len(matrice) != 0:
        indice = m.compute(matrice)
        i_min = min((len(old_pos_order), len(to_assin)))
        indice = indice[:i_min] # on suprime le 0 pading
        for (old, new) in indice:
            pt_old = old_pos_order[old]
            pt_new = to_assin[new]
            old_pos_traite.append(pt_old)
            new_pos_traite.append(pt_new)
            rep[pt_new] = oldPos[pt_old]
            pt_new.nb_of_use = pt_old.nb_of_use + 1
                             

    for elm in to_assin:
        if elm not in new_pos_traite:
            if len(rep) == 0:
                rep[elm] = 1
            else:
                rep[elm] = max(rep.values()) + 1
                # print ("\n")
                # print("New element ! ! ! " + str(rep))
                # print("oldpos : " + str(oldPos))
                # print(to_assin)
                # print("Frame : " + str(frame))
            new_pos_traite.append(elm)

    for elm in oldPos.keys():
        if elm not in old_pos_traite and elm.nb_of_use > 5:
            rep[elm] = oldPos[elm]
            old_pos_traite.append(elm)
    
    return rep


def dist((A, B)):
    return A.dist(B)


res = make_pred(row_data, make_alea(0.001))
print(res.shape)
print(compute_clear(data_ref, res, 1))


(13231, 39)


(0.9787506593323789, 0.0, 0.6326855928343708, 0.6141210157486248)


In [5]:
# sauvegarde les donnée
save_data(res, PATH+"res_tmp.txt")

In [42]:
a = {"a" : 1}
print(len(a))

1


In [48]:
b = a.keys()
print(b)

['a']
